# DESI Iron
Determine how many galaxies were observed in DESI Iron that are good TF targets by finding the number of unique galaxies observed with observations at the center and at .4R.

In [ ]:
## Module imports and constant definitions 
from astropy.table import unique, Table
from astropy.coordinates import SkyCoord
from astropy.io import fits 

import astropy.units as u
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt 

from tqdm.auto import tqdm

import os

import requests

from astropy.wcs import WCS
from astropy.visualization.wcsaxes import SphericalCircle

h = 1
H0 = 100*h
c = 3e5
q0 = 0.2
V0 = 2.5 # 0-point of TFR

### Iron Data
We read in the Iron data to use throughout the notebook. Commented out below is the Fuji data, to use to check against when necessary. 

In [ ]:
tiron = Table.read('/global/cfs/projectdirs/desi/science/td/pv/desi_pv_tf_iron_healpix.fits')
#tiron = Table.read('/global/cfs/projectdirs/desi/science/td/pv/desi_pv_tf_fuji_healpix.fits')

tiron[:5]

### SGA
We read in the SGA to use throughout the notebook. 

In [ ]:
SGA = Table.read('/global/cfs/cdirs/cosmo/data/sga/2020/SGA-2020.fits', 'ELLIPSE')

## Create a dictionary of SGA IDs to find row indices 
SGA_dict = {}
for i in range(len(SGA)):
    SGA_dict[SGA['SGA_ID'][i]] = i

SGA[:5]

## Separation between Galaxies and Observations
Find all targets on each galaxy, and then calculate distance between SGA center coordinates and observation coordinates. 

In [ ]:
tiron['SKY_FIBER_DIST'] = 0.
tiron['SKY_FIBER_DIST_R26'] = 0.

## For each SGA galaxy that has 1+ observations, calculate distance for all of its targets
for sga_id in tqdm(np.unique(tiron['SGA_ID'])):
    
    ## Identify all galaxy targets on this galaxy
    obs_idx = tiron['SGA_ID'] == sga_id
    
    ## Find galaxy index in SGA catalog
    sga_idx = SGA_dict[sga_id]
    
    ## Calculate distance between each observation and the center of the galaxy
    SGA_coords = SkyCoord(ra=SGA['RA'][sga_idx], 
                          dec=SGA['DEC'][sga_idx], 
                          unit=u.degree)
    target_coords = SkyCoord(ra=tiron['RA'][obs_idx], 
                             dec=tiron['DEC'][obs_idx], 
                             unit=u.degree)
    sep2d = target_coords.separation(SGA_coords)
    
    ## Add the distance to the tiron table
    tiron['SKY_FIBER_DIST'][obs_idx] = sep2d
    ## Add the distance in R26 to the tiron table 
    tiron['SKY_FIBER_DIST_R26'][obs_idx] = 2*sep2d.to('arcmin')/(SGA['D26'][sga_idx]*u.arcmin)

## Isolate the centers to be those measurements where the distance is <.1*R26
centers_boolean = tiron['SKY_FIBER_DIST_R26'] < 0.1

## Sort observations into center and axis observations 
iron_centers = tiron[centers_boolean]
iron_axis = tiron[~centers_boolean]

## Cleaning Iron Center Observations

Only keep those observations with
 * `DELTACHI2` > 25
 * `ZWARN` == 0

In [ ]:
good_centers = iron_centers[(iron_centers['DELTACHI2'] > 25) & (iron_centers['ZWARN'] == 0)]

## Check for multiple good center observations 
unique_ids, counts = np.unique(good_centers['SGA_ID'], return_counts=True)

If there's at least one good center observation, set the galaxy's redshift.

In [ ]:
SGA['Z_DESI'] = np.nan
SGA['ZERR_DESI'] = np.nan

weights = 1./(good_centers['ZERR']**2)

for sga_id in np.unique(good_centers['SGA_ID']):
    
    ## Find all the center observations of this galaxy
    obs_idx = good_centers['SGA_ID'] == sga_id
    
    ## Find the row in SGA for this galaxy
    SGA_idx = SGA_dict[sga_id]
    
    # Set the redshift of this galaxy to be weighted average of all good center observation redshifts
    SGA['Z_DESI'][SGA_idx] = np.average(good_centers['Z'][obs_idx], 
                                        weights=weights[obs_idx])
    SGA['ZERR_DESI'][SGA_idx] = np.sqrt(1./np.sum(weights[obs_idx]))

Determine how many observations at $.4R$ we have. 

In [ ]:
## Include all observations where .38R < distance < .42R in our .4R values
r0p4 = iron_axis[(iron_axis['SKY_FIBER_DIST_R26'] > 0.38) & (iron_axis['SKY_FIBER_DIST_R26'] < 0.42)]

## Count how many of these are unique values
unique_centers = np.unique(good_centers['SGA_ID'])
unique_r0p4 = np.unique(r0p4['SGA_ID'])

centers_and_p4s = []
for i in unique_r0p4: 
    if i in unique_centers: 
        centers_and_p4s.append(i)

## Output the number of observations that are generally good for TF fitting
print(len(centers_and_p4s)," unique galaxies with center and .4R observations")

## Find the percentage of all Iron galaxies that are generally good for TF fitting
num_iron = len(np.unique(tiron['SGA_ID']))
percentage_TF = 100*((len(centers_and_p4s))/num_iron)
print(percentage_TF,"% of all Iron galaxies")

## Cluster Membership
Following Cosmicflows4 (Kourkchi et al. 2020), cluster membership is defined as
- $R_p < 1.5R_{2t}$ and $v < V_c \pm 3\sigma_p$
- $1.5R_{2t} \leq R_p < 3R_{2t}$ and $v < V_c \pm 2\sigma_p$

where $R_p$ is the projected distance from the cluster center, $R_{2t}$ is the cluster projected second turnaround radius, $\sigma_p$ is the projected velocity dispersion of the cluster, and $V_c$ is the average heliocentric radial velocity of the cluster.

### Tully et. al. (2015) Table 3

In [ ]:
hdu = fits.open("../Tully15-Table3.fits")
table3 = Table(hdu[1].data)
table3.rename_column("<Vcmba>","VMod")
hdu.close()

table3_dict = {}
for i in range(len(table3)):
    table3_dict[table3['Nest'][i]] = i

table3[:5]

In [ ]:
## Check a suspicious value from later on in the notebook
table3[table3['Nest'] == 100526]

In [ ]:
## Continue the suspicious value check 
cluster_coord = SkyCoord(table3['SGLON'][table3['Nest'] == 100526]*u.degree, 
                         table3['SGLAT'][table3['Nest'] == 100526]*u.degree, 
                         frame='supergalactic')
print(cluster_coord.transform_to('icrs'))

### Tully et. al (2013) Table 2

In [ ]:
hdu = fits.open("../Tully13-Table2.fit")
table2 = Table(hdu[1].data)
hdu.close()

table2[:5]

Loop through the SGA table to do just the rows that are for IDs we want from centers_and_p4s. Collect them into a smaller table. 

In [ ]:
SGA.add_index("SGA_ID")
TF_SGA_cp4 = SGA.loc[centers_and_p4s]

TF_SGA_cp4[:5]

In [ ]:
## Cluster Membership function. Takes argument of nest ID for the target cluster.

def cluster_membership(nest_id):
    c_4r_in_cluster = []
    
    ## Set active row for each nest number and base values 
    active_row = table3_dict[nest_id]
    R2t = table3["R2t"][active_row]
    sigma = table3["sigP"][active_row]
    
    ## Find the coordinates for each cluster
    cluster_coords = SkyCoord(table3["SGLON"][active_row]*u.degree, 
                       table3["SGLAT"][active_row]*u.degree, 
                       frame='supergalactic')
    
    group_coords = SkyCoord(table2['SGLON']*u.degree, 
                        table2['SGLAT']*u.degree, 
                        frame='supergalactic')
    
    ## Match cluster to group coordinates 
    idx, d2d, d3d = cluster_coords.match_to_catalog_sky(group_coords)
    v = table2["__HV_"][idx]
    
    ## Match SGA coordinates from unique galaxies to nearest cluster
    SGA_coords = SkyCoord(TF_SGA_cp4['RA'], TF_SGA_cp4['DEC'], unit='deg')
    sep = cluster_coords.separation(SGA_coords)

    ## Convert R2t to an angle 
    R2t_angle = (R2t/(v/H0))*u.radian
    
    SGA_in_cluster1 = (sep < 1.5*R2t_angle) & (TF_SGA_cp4["Z_DESI"]*c > v - 3*sigma) & (TF_SGA_cp4["Z_DESI"]*c < v + 3*sigma)
    SGA_in_cluster2 = (sep >= 1.5*R2t_angle) & (sep < 3*R2t_angle) & (TF_SGA_cp4["Z_DESI"]*c > v - 2*sigma) & (TF_SGA_cp4["Z_DESI"]*c < v + 2*sigma)
    
    SGA_in_cluster = SGA_in_cluster1 | SGA_in_cluster2
    
    ## Keep observations that are within nest cluster 
    SGA_ID_in_cluster = TF_SGA_cp4["SGA_ID"][SGA_in_cluster]
    
    ## Gather centers and axes in the cluster
    centers_in_cluster = good_centers[np.in1d(good_centers['SGA_ID'], SGA_ID_in_cluster)]
    axis_in_cluster = iron_axis[np.in1d(iron_axis['SGA_ID'], SGA_ID_in_cluster)]
    
    c_4r_in_cluster.append(SGA_ID_in_cluster)
    return(c_4r_in_cluster, SGA_in_cluster, SGA_ID_in_cluster, centers_in_cluster, axis_in_cluster, cluster_coords, v, sep)

## Abell-2151 Explicit Filtering

In [ ]:
(c_4r_in_Abell, SGA_in_Abell, SGA_ID_in_Abell, centers_in_Abell, axis_in_Abell, Abell_coords, v_Abell, sep_Abell)

In [ ]:
## make a bar graph of the angular separation 
plt.hist(sep[SGA_in_Abell].to_value('degree'), bins=np.arange(0,5, 0.5))
plt.xlabel('SGA-Abell-2151 Angular Separation [deg]')
plt.ylabel('number of galaxies');

In [ ]:
## plot the physical locations and the redshifts of the cluster

plt.figure(figsize=(15,5), tight_layout=True)

plt.subplot(131)
plt.plot(centers_in_Abell['TARGET_RA'], centers_in_Abell['TARGET_DEC'], '.')
plt.plot(Abell_coords.transform_to('icrs').ra.deg, Abell_coords.transform_to('icrs').dec.deg, 'kx', ms=10, mew=5)
plt.xlabel(r'$\alpha$ [deg]')
plt.ylabel(r'$\delta$ [deg]')

plt.subplot(132)
plt.plot(centers_in_Abell['Z'], centers_in_Abell['TARGET_DEC'], '.')
plt.plot(v/c, Abell_coords.transform_to('icrs').dec.deg, 'kx', ms=10, mew=5)
plt.xlabel('z')
plt.ylabel(r'$\delta$ [deg]')

plt.subplot(133)
plt.hist(centers_in_Abell['Z'], bins=np.arange(0.02, .06, 0.01))
plt.vlines(v/c, 0, 100, colors='k', linestyles='dotted')
plt.xlabel('redshift')
plt.ylabel('number of galaxies')
plt.ylim(ymax=95);

## Virgo Explicit Filtering

In [ ]:
(c_4r_in_Virgo, SGA_in_Virgo, SGA_ID_in_Virgo, centers_in_Virgo, axis_in_Virgo, Virgo_coords, v_Virgo, sep_Virgo) = = cluster_membership(100002)

In [ ]:
## make a bar graph of the angular separation 
plt.hist(sep[SGA_in_Virgo].to_value('degree'), bins=np.arange(0,5, 0.5))
plt.xlabel('SGA-Virgo Angular Separation [deg]')
plt.ylabel('number of galaxies');

In [ ]:
## plot the physical locations and the redshifts of the cluster

plt.figure(figsize=(15,5), tight_layout=True)

plt.subplot(131)
plt.plot(centers_in_Virgo['TARGET_RA'], centers_in_Virgo['TARGET_DEC'], '.')
plt.plot(Virgo_coords.transform_to('icrs').ra.deg, Virgo_coords.transform_to('icrs').dec.deg, 'kx', ms=10, mew=5)
plt.xlabel(r'$\alpha$ [deg]')
plt.ylabel(r'$\delta$ [deg]')

plt.subplot(132)
plt.plot(centers_in_Virgo['Z'], centers_in_Virgo['TARGET_DEC'], '.')
plt.plot(v/c, Virgo_coords.transform_to('icrs').dec.deg, 'kx', ms=10, mew=5)
plt.xlabel('z')
plt.ylabel(r'$\delta$ [deg]')

plt.subplot(133)
plt.hist(centers_in_Virgo['Z'], bins=np.arange(0.02, .06, 0.01))
plt.vlines(v/c, 0, 100, colors='k', linestyles='dotted')
plt.xlabel('redshift')
plt.ylabel('number of galaxies')
plt.ylim(ymax=95);

## Abell-2151 Filtering

In [ ]:
## make a bar graph of the angular separation 
plt.hist(sep[SGA_in_cluster].to_value('degree'), bins=np.arange(0,5, 0.5))
plt.xlabel('SGA-Abell-2151 Angular Separation [deg]')
plt.ylabel('number of galaxies');

In [ ]:
## plot the physical locations and the redshifts of the cluster

plt.figure(figsize=(15,5), tight_layout=True)

plt.subplot(131)
plt.plot(centers_in_cluster['TARGET_RA'], centers_in_cluster['TARGET_DEC'], '.')
plt.plot(cluster_coords.transform_to('icrs').ra.deg, cluster_coords.transform_to('icrs').dec.deg, 'kx', ms=10, mew=5)
plt.xlabel(r'$\alpha$ [deg]')
plt.ylabel(r'$\delta$ [deg]')

plt.subplot(132)
plt.plot(centers_in_cluster['Z'], centers_in_cluster['TARGET_DEC'], '.')
plt.plot(v/c, cluster_coords.transform_to('icrs').dec.deg, 'kx', ms=10, mew=5)
plt.xlabel('z')
plt.ylabel(r'$\delta$ [deg]')

plt.subplot(133)
plt.hist(centers_in_cluster['Z'], bins=np.arange(0.02, .06, 0.01))
plt.vlines(v/c, 0, 100, colors='k', linestyles='dotted')
plt.xlabel('redshift')
plt.ylabel('number of galaxies')
plt.ylim(ymax=95);

In [ ]:
axis_SGAids, axis_counts = np.unique(axis_in_cluster['SGA_ID'], return_counts=True)
center_SGAids, center_counts = np.unique(centers_in_cluster['SGA_ID'], return_counts=True)

counts = []

for sga_id in SGA_ID_in_cluster:
    
    center_count = 0
    axis_count = 0
    
    if sga_id in center_SGAids:
        
        center_count = center_counts[center_SGAids == sga_id]
        
    if sga_id in axis_SGAids:
        
        axis_count = axis_counts[axis_SGAids == sga_id]
        
    count = center_count + axis_count
    
    if count > 1:
        
        counts.append(count)
        

plt.figure(tight_layout=True)

plt.hist(np.array(counts), bins=np.arange(2,15))

plt.xlabel('Observations per SGA_ID in Abell-2151')
plt.ylabel('count');

### Calculate the rotational velocity

In [ ]:
axis_in_cluster['SKY_FIBER_DIST'] = 0.
axis_in_cluster['SKY_FIBER_DIST_R26'] = 0.
axis_in_cluster['V_ROT'] = np.nan
axis_in_cluster['V_ROT_ERR'] = np.nan


# For each SGA galaxy that has at least one center observation, calculate the 
# distance for all of that galaxy's targets
for sga_gal in np.unique(centers_in_cluster['SGA_ID']):
    
    # Identify all galaxy targets on this galaxy
    obs_idx = axis_in_cluster['SGA_ID'] == sga_gal
    
    # Find galaxy index in SGA catalog
    sga_idx = SGA_dict[sga_gal]
    
    #---------------------------------------------------------------------------
    # Calculate distance between each observation and the center
    #---------------------------------------------------------------------------
    center_coords = SkyCoord(ra=SGA['RA'][sga_idx], 
                             dec=SGA['DEC'][sga_idx], 
                             unit=u.degree)
    target_coords = SkyCoord(ra=axis_in_cluster['RA'][obs_idx], 
                             dec=axis_in_cluster['DEC'][obs_idx], 
                             unit=u.degree)
    
    sep2d = target_coords.separation(center_coords)
    
    axis_in_cluster['SKY_FIBER_DIST'][obs_idx] = sep2d
    axis_in_cluster['SKY_FIBER_DIST_R26'][obs_idx] = 2*sep2d.to('arcmin')/(SGA['D26'][sga_idx]*u.arcmin)
    #---------------------------------------------------------------------------
    
    
    #---------------------------------------------------------------------------
    # Calculate rotational velocity
    #---------------------------------------------------------------------------
    # Use the average redshift of all center observations for the systemic velocity
    z_center = np.mean(SGA['Z_DESI'][sga_idx])
    z_err_center2 = SGA['ZERR_DESI'][sga_idx]**2

    # Calculate rotational velocity for all observations of the galaxy
    axis_in_cluster['V_ROT'][obs_idx] = c*(axis_in_cluster['Z'][obs_idx] - z_center)
    axis_in_cluster['V_ROT_ERR'][obs_idx] = c*np.sqrt(axis_in_cluster['ZERR'][obs_idx]**2 + z_err_center2)
    #---------------------------------------------------------------------------
    
    
    #---------------------------------------------------------------------------
    # Correct rotational velocities for inclination angle
    #---------------------------------------------------------------------------
    cosi2 = (SGA['BA'][sga_idx]**2 - q0**2)/(1 - q0**2)
    
    # Galaxies with b/a < q0
    if cosi2 < 0:
        cosi2 = 0
    
    axis_in_cluster['V_ROT'][obs_idx] /= np.sin(np.arccos(np.sqrt(cosi2)))
    #---------------------------------------------------------------------------

In [ ]:
plt.figure(tight_layout=True)

plt.hist(np.abs(axis_in_cluster['V_ROT']), bins=np.linspace(0, 1000, 100))

plt.xlabel('$V_{rot}$ [km/s]')
plt.ylabel('number of observations');

### Cut for Abell-2151 galaxies suitable for calibrating the TFR

Requirements:
 * $10 < V_{rot} < 1000$ km/s at $0.33R_{26}$
 * $\Delta V / V_{min} \leq 5$
 * $i > 45^\circ$
 * Spiral-type morphology
 * Passes visual inspection

#### Velocity Cut

In [ ]:
r0p4 = (axis_in_cluster['SKY_FIBER_DIST_R26'] > 0.38) & (axis_in_cluster['SKY_FIBER_DIST_R26'] < 0.42)

Vgood = (np.abs(axis_in_cluster['V_ROT']) < 1000) & (np.abs(axis_in_cluster['V_ROT']) > 10)

good_axis_in_cluster = axis_in_cluster[r0p4 & Vgood]

print(len(good_axis_in_cluster), len(np.unique(good_axis_in_cluster['SGA_ID'])))

#### Relative Velocity Cut

In [ ]:
good_deltaV = np.ones(len(good_axis_in_cluster), dtype=bool)

for sga_id in np.unique(good_axis_in_cluster['SGA_ID']):
    
    # Identify all galaxy targets on this galaxy
    obs_idx = good_axis_in_cluster['SGA_ID'] == sga_id
    
    n_obs = np.sum(obs_idx)
    
    if n_obs > 1:
        
        Vmin = np.min(np.abs(good_axis_in_cluster['V_ROT'][obs_idx]))
        Vmax = np.max(np.abs(good_axis_in_cluster['V_ROT'][obs_idx]))
        
        v_norm_min = np.abs(good_axis_in_cluster['V_ROT'][obs_idx])/Vmin
        v_norm_max = np.abs(good_axis_in_cluster['V_ROT'][obs_idx])/Vmax
        
        diff_matrix = np.abs(good_axis_in_cluster['V_ROT'][obs_idx]).reshape(n_obs, 1) - np.abs(good_axis_in_cluster['V_ROT'][obs_idx]).reshape(1, n_obs)
        
        diff_matrix_norm = diff_matrix/Vmin
        
        if np.any(np.abs(diff_matrix_norm) > 5.):
            
            '''
            print(sga_id)
            print(diff_matrix_norm)
            print(1/v_norm_min.data)
            print(v_norm_max.data)
            print(good_axis_inComa[['TARGETID', 'V_ROT', 'PVTYPE', 'FILENAME', 'DELTACHI2', 'ZWARN']][obs_idx].pprint(max_width=-1))
            ''';
            
            # Remove all observations with DELTACHI2 < 25
            # Note: This also typically removes observations with ZWARN != 0
            deltachi2_idx = good_axis_in_cluster['DELTACHI2'] >= 25
            
            good_deltaV[obs_idx & ~deltachi2_idx] = False
            
            good_obs_idx = obs_idx & deltachi2_idx
            
            n_obs_good = np.sum(good_obs_idx)
            
            # Check to make sure that, if there are still multiple observations, they all satisfy our relative velocity criteria
            if n_obs_good > 1:
                
                Vmin = np.min(np.abs(good_axis_in_cluster['V_ROT'][good_obs_idx]))
                
                diff_matrix = np.abs(good_axis_in_cluster['V_ROT'][good_obs_idx]).reshape(n_obs_good, 1) - np.abs(good_axis_in_cluster['V_ROT'][good_obs_idx]).reshape(1, n_obs_good)
                
                diff_matrix_norm = diff_matrix/Vmin
                
                if np.any(np.abs(diff_matrix_norm) > 5.):
                    '''
                    print(sga_id)
                    print(diff_matrix_norm)
                    print(good_axis_inComa[['TARGETID', 'V_ROT', 'PVTYPE', 'FILENAME', 'DELTACHI2', 'ZWARN']][obs_idx].pprint(max_width=-1))
                    ''';
                    # Set all of these so that we don't look at this galaxy
                    good_deltaV[good_obs_idx] = False

In [ ]:
good_deltaV_axis_in_cluster = good_axis_in_cluster[good_deltaV]

print(len(good_deltaV_axis_in_cluster), len(np.unique(good_deltaV_axis_in_cluster['SGA_ID'])))

#### Inclination Angle Cut

In [ ]:
SGA['cosi2'] = (SGA['BA']**2 - q0**2)/(1 - q0**2)
SGA['cosi2'][SGA['cosi2'] < 0] = 0

good_deltaV_axis_in_cluster['iSGA'] = -1

for i in range(len(good_deltaV_axis_in_cluster)):
    
    # Find galaxy in SGA
    sga_idx = SGA_dict[good_deltaV_axis_in_cluster['SGA_ID'][i]]
    
    good_deltaV_axis_in_cluster['iSGA'][i] = sga_idx
    
good_deltaV_axis_in_cluster['cosi2'] = SGA['cosi2'][good_deltaV_axis_in_cluster['iSGA']]

In [ ]:
i_min = 45. # degrees

cosi2_max = np.cos(i_min*np.pi/180.)**2

edge = good_deltaV_axis_in_cluster['cosi2'] <= cosi2_max

good_edge_axis_in_cluster = good_deltaV_axis_in_cluster[edge]

print(len(good_edge_axis_in_cluster), len(np.unique(good_edge_axis_in_cluster['SGA_ID'])))

In [ ]:
plt.figure(tight_layout=True)

plt.hist(np.arccos(np.sqrt(good_edge_axis_in_cluster['cosi2']))*180/np.pi, bins=np.linspace(0, 90, 10))

plt.xlabel('inclination angle [deg]')
plt.ylabel('number of observations');

#### Morphology Cut

In [ ]:
good_edge_axis_in_cluster['MORPHTYPE'] = SGA['MORPHTYPE'][good_edge_axis_in_cluster['iSGA']]

In [ ]:
spirals = np.zeros(len(good_edge_axis_in_cluster), dtype=bool)

for i in range(len(good_edge_axis_in_cluster)):
    
    try:    
        if (good_edge_axis_in_cluster['MORPHTYPE'][i][0] == 'S') and (good_edge_axis_in_cluster['MORPHTYPE'][i][:2] != 'S0'):
            spirals[i] = True
    except IndexError:
        print(good_edge_axis_in_cluster['MORPHTYPE'][i])

good_edge_spirals_axis_in_cluster = good_edge_axis_in_cluster[spirals]

print(len(good_edge_spirals_axis_in_cluster), len(np.unique(good_edge_spirals_axis_in_cluster['SGA_ID'])))

Stopped at visual inspection cut because the number of galaxies dropped below 20.

## Virgo Filtering

In [ ]:
(c_4r_in_cluster, SGA_in_cluster, SGA_ID_in_cluster, centers_in_cluster, axis_in_cluster, cluster_coords, v, sep) = cluster_membership(100002)

In [ ]:
## make a bar graph of the angular separation 
plt.hist(sep[SGA_in_cluster].to_value('degree'), bins=np.arange(15,25, 0.5))
plt.xlabel('SGA-Abell-2151 Angular Separation [deg]')
plt.ylabel('number of galaxies');

In [ ]:
# plt.plot(centers_in_cluster['TARGET_RA'], centers_in_cluster['TARGET_DEC'], '.')
with open('virgo.txt', 'w') as f:
    for ra, dec in centers_in_cluster['TARGET_RA', 'TARGET_DEC']:
        f.write(f'{ra} {dec}\n')

In [ ]:
## plot the physical locations and the redshifts of the cluster

plt.figure(figsize=(15,5), tight_layout=True)

plt.subplot(131)
plt.plot(centers_in_cluster['TARGET_RA'], centers_in_cluster['TARGET_DEC'], '.')
plt.plot(cluster_coords.transform_to('icrs').ra.deg, cluster_coords.transform_to('icrs').dec.deg, 'kx', ms=10, mew=5)
plt.xlabel(r'$\alpha$ [deg]')
plt.ylabel(r'$\delta$ [deg]')

plt.subplot(132)
plt.plot(centers_in_cluster['Z'], centers_in_cluster['TARGET_DEC'], '.')
plt.plot(v/c, cluster_coords.transform_to('icrs').dec.deg, 'kx', ms=10, mew=5)
plt.xlabel('z')
plt.ylabel(r'$\delta$ [deg]')

plt.subplot(133)
plt.hist(centers_in_cluster['Z'], bins=np.arange(0.001, .01, 0.0005))
plt.vlines(v/c, 0, 100, colors='k', linestyles='dotted')
plt.xlabel('redshift')
plt.ylabel('number of galaxies')
plt.ylim(ymax=95);

In [ ]:
axis_SGAids, axis_counts = np.unique(axis_in_cluster['SGA_ID'], return_counts=True)
center_SGAids, center_counts = np.unique(centers_in_cluster['SGA_ID'], return_counts=True)

counts = []

for sga_id in SGA_ID_in_cluster:
    
    center_count = 0
    axis_count = 0
    
    if sga_id in center_SGAids:
        
        center_count = center_counts[center_SGAids == sga_id]
        
    if sga_id in axis_SGAids:
        
        axis_count = axis_counts[axis_SGAids == sga_id]
        
    count = center_count + axis_count
    
    if count > 1:
        
        counts.append(count)
        

plt.figure(tight_layout=True)

plt.hist(np.array(counts), bins=np.arange(2,15))

plt.xlabel('Observations per SGA_ID in Abell-2151')
plt.ylabel('count');

### Calculate the rotational velocity

In [ ]:
axis_in_cluster['SKY_FIBER_DIST'] = 0.
axis_in_cluster['SKY_FIBER_DIST_R26'] = 0.
axis_in_cluster['V_ROT'] = np.nan
axis_in_cluster['V_ROT_ERR'] = np.nan


# For each SGA galaxy that has at least one center observation, calculate the 
# distance for all of that galaxy's targets
for sga_gal in np.unique(centers_in_cluster['SGA_ID']):
    
    # Identify all galaxy targets on this galaxy
    obs_idx = axis_in_cluster['SGA_ID'] == sga_gal
    
    # Find galaxy index in SGA catalog
    sga_idx = SGA_dict[sga_gal]
    
    #---------------------------------------------------------------------------
    # Calculate distance between each observation and the center
    #---------------------------------------------------------------------------
    center_coords = SkyCoord(ra=SGA['RA'][sga_idx], 
                             dec=SGA['DEC'][sga_idx], 
                             unit=u.degree)
    target_coords = SkyCoord(ra=axis_in_cluster['RA'][obs_idx], 
                             dec=axis_in_cluster['DEC'][obs_idx], 
                             unit=u.degree)
    
    sep2d = target_coords.separation(center_coords)
    
    axis_in_cluster['SKY_FIBER_DIST'][obs_idx] = sep2d
    axis_in_cluster['SKY_FIBER_DIST_R26'][obs_idx] = 2*sep2d.to('arcmin')/(SGA['D26'][sga_idx]*u.arcmin)
    #---------------------------------------------------------------------------
    
    
    #---------------------------------------------------------------------------
    # Calculate rotational velocity
    #---------------------------------------------------------------------------
    # Use the average redshift of all center observations for the systemic velocity
    z_center = np.mean(SGA['Z_DESI'][sga_idx])
    z_err_center2 = SGA['ZERR_DESI'][sga_idx]**2

    # Calculate rotational velocity for all observations of the galaxy
    axis_in_cluster['V_ROT'][obs_idx] = c*(axis_in_cluster['Z'][obs_idx] - z_center)
    axis_in_cluster['V_ROT_ERR'][obs_idx] = c*np.sqrt(axis_in_cluster['ZERR'][obs_idx]**2 + z_err_center2)
    #---------------------------------------------------------------------------
    
    
    #---------------------------------------------------------------------------
    # Correct rotational velocities for inclination angle
    #---------------------------------------------------------------------------
    cosi2 = (SGA['BA'][sga_idx]**2 - q0**2)/(1 - q0**2)
    
    # Galaxies with b/a < q0
    if cosi2 < 0:
        cosi2 = 0
    
    axis_in_cluster['V_ROT'][obs_idx] /= np.sin(np.arccos(np.sqrt(cosi2)))
    #---------------------------------------------------------------------------

In [ ]:
plt.figure(tight_layout=True)

plt.hist(np.abs(axis_in_cluster['V_ROT']), bins=np.linspace(0, 1000, 100))

plt.xlabel('$V_{rot}$ [km/s]')
plt.ylabel('number of observations');

### Cut for Virgo galaxies suitable for calibrating the TFR

Requirements:
 * $10 < V_{rot} < 1000$ km/s at $0.33R_{26}$
 * $\Delta V / V_{min} \leq 5$
 * $i > 45^\circ$
 * Spiral-type morphology
 * Passes visual inspection

### Velocity cut

In [ ]:
r0p4 = (axis_in_cluster['SKY_FIBER_DIST_R26'] > 0.38) & (axis_in_cluster['SKY_FIBER_DIST_R26'] < 0.42)

Vgood = (np.abs(axis_in_cluster['V_ROT']) < 1000) & (np.abs(axis_in_cluster['V_ROT']) > 10)

good_axis_in_cluster = axis_in_cluster[r0p4 & Vgood]

print(len(good_axis_in_cluster), len(np.unique(good_axis_in_cluster['SGA_ID'])))

### Relative velocity cut

In [ ]:
good_deltaV = np.ones(len(good_axis_in_cluster), dtype=bool)

for sga_id in np.unique(good_axis_in_cluster['SGA_ID']):
    
    # Identify all galaxy targets on this galaxy
    obs_idx = good_axis_in_cluster['SGA_ID'] == sga_id
    
    n_obs = np.sum(obs_idx)
    
    if n_obs > 1:
        
        Vmin = np.min(np.abs(good_axis_in_cluster['V_ROT'][obs_idx]))
        Vmax = np.max(np.abs(good_axis_in_cluster['V_ROT'][obs_idx]))
        
        v_norm_min = np.abs(good_axis_in_cluster['V_ROT'][obs_idx])/Vmin
        v_norm_max = np.abs(good_axis_in_cluster['V_ROT'][obs_idx])/Vmax
        
        diff_matrix = np.abs(good_axis_in_cluster['V_ROT'][obs_idx]).reshape(n_obs, 1) - np.abs(good_axis_in_cluster['V_ROT'][obs_idx]).reshape(1, n_obs)
        
        diff_matrix_norm = diff_matrix/Vmin
        
        if np.any(np.abs(diff_matrix_norm) > 5.):
            
            '''
            print(sga_id)
            print(diff_matrix_norm)
            print(1/v_norm_min.data)
            print(v_norm_max.data)
            print(good_axis_inComa[['TARGETID', 'V_ROT', 'PVTYPE', 'FILENAME', 'DELTACHI2', 'ZWARN']][obs_idx].pprint(max_width=-1))
            ''';
            
            # Remove all observations with DELTACHI2 < 25
            # Note: This also typically removes observations with ZWARN != 0
            deltachi2_idx = good_axis_in_cluster['DELTACHI2'] >= 25
            
            good_deltaV[obs_idx & ~deltachi2_idx] = False
            
            good_obs_idx = obs_idx & deltachi2_idx
            
            n_obs_good = np.sum(good_obs_idx)
            
            # Check to make sure that, if there are still multiple observations, they all satisfy our relative velocity criteria
            if n_obs_good > 1:
                
                Vmin = np.min(np.abs(good_axis_in_cluster['V_ROT'][good_obs_idx]))
                
                diff_matrix = np.abs(good_axis_in_cluster['V_ROT'][good_obs_idx]).reshape(n_obs_good, 1) - np.abs(good_axis_in_cluster['V_ROT'][good_obs_idx]).reshape(1, n_obs_good)
                
                diff_matrix_norm = diff_matrix/Vmin
                
                if np.any(np.abs(diff_matrix_norm) > 5.):
                    '''
                    print(sga_id)
                    print(diff_matrix_norm)
                    print(good_axis_inComa[['TARGETID', 'V_ROT', 'PVTYPE', 'FILENAME', 'DELTACHI2', 'ZWARN']][obs_idx].pprint(max_width=-1))
                    ''';
                    # Set all of these so that we don't look at this galaxy
                    good_deltaV[good_obs_idx] = False

In [ ]:
good_deltaV_axis_in_cluster = good_axis_in_cluster[good_deltaV]

print(len(good_deltaV_axis_in_cluster), len(np.unique(good_deltaV_axis_in_cluster['SGA_ID'])))

### Inclination angle cut

In [ ]:
SGA['cosi2'] = (SGA['BA']**2 - q0**2)/(1 - q0**2)
SGA['cosi2'][SGA['cosi2'] < 0] = 0

good_deltaV_axis_in_cluster['iSGA'] = -1

for i in range(len(good_deltaV_axis_in_cluster)):
    
    # Find galaxy in SGA
    sga_idx = SGA_dict[good_deltaV_axis_in_cluster['SGA_ID'][i]]
    
    good_deltaV_axis_in_cluster['iSGA'][i] = sga_idx
    
good_deltaV_axis_in_cluster['cosi2'] = SGA['cosi2'][good_deltaV_axis_in_cluster['iSGA']]

In [ ]:
i_min = 45. # degrees

cosi2_max = np.cos(i_min*np.pi/180.)**2

edge = good_deltaV_axis_in_cluster['cosi2'] <= cosi2_max

good_edge_axis_in_cluster = good_deltaV_axis_in_cluster[edge]

print(len(good_edge_axis_in_cluster), len(np.unique(good_edge_axis_in_cluster['SGA_ID'])))

In [ ]:
plt.figure(tight_layout=True)

plt.hist(np.arccos(np.sqrt(good_edge_axis_in_cluster['cosi2']))*180/np.pi, bins=np.linspace(0, 90, 10))

plt.xlabel('inclination angle [deg]')
plt.ylabel('number of observations');

### Morphology cut

In [ ]:
good_edge_axis_in_cluster['MORPHTYPE'] = SGA['MORPHTYPE'][good_edge_axis_in_cluster['iSGA']]

In [ ]:
spirals = np.zeros(len(good_edge_axis_in_cluster), dtype=bool)

for i in range(len(good_edge_axis_in_cluster)):
    
    try:    
        if (good_edge_axis_in_cluster['MORPHTYPE'][i][0] == 'S') and (good_edge_axis_in_cluster['MORPHTYPE'][i][:2] != 'S0'):
            spirals[i] = True
    except IndexError:
        print(good_edge_axis_in_cluster['MORPHTYPE'][i])

good_edge_spirals_axis_in_cluster = good_edge_axis_in_cluster[spirals]

print(len(good_edge_spirals_axis_in_cluster), len(np.unique(good_edge_spirals_axis_in_cluster['SGA_ID'])))

### Visual Inspection Cut

In [ ]:
def get_cutout(targetid, ra, dec, size, verbose=False):
    """Grab and cache legacy survey cutouts.
    
    Parameters
    ----------
    targetid : int
        DESI target ID.
    ra : float
        Right ascension (degrees).
    dec : float
        Declination (degrees).
    verbose : bool
        Add some status messages if true.
        
    Returns
    -------
    img_name : str
        Name of JPG cutout file written after query.
    w : astropy.wcs.WCS
        World coordinate system for the image.
    """
    # Either load an existing image or download a cutout.
    img_name = 'cache/iron_{}.jpg'.format(targetid)
    
    if os.path.exists(img_name):
        if verbose:
            print('{} exists.'.format(img_name))
    else:
        img_url = 'https://www.legacysurvey.org/viewer/cutout.jpg?ra={}&dec={}&zoom=14&layer=ls-dr9&size={}&sga'.format(ra, dec, size)
        if verbose:
            print('Get {}'.format(img_url))
            
        with open(img_name, 'wb') as handle: 
            response = requests.get(img_url, stream=True) 
            if not response.ok: 
                print(response) 
            for block in response.iter_content(1024): 
                if not block: 
                    break 
                handle.write(block)
                
    # Set up the WCS.
    wcs_input_dict = {
        'CTYPE1': 'RA---TAN',
        'CUNIT1': 'deg',
        'CDELT1': -0.262/3600,
        'CRPIX1': size/2 + 0.5,
        'CRVAL1': ra,
        'NAXIS1': size,
        'CTYPE2': 'DEC--TAN',
        'CUNIT2': 'deg',
        'CDELT2': 0.262/3600,
        'CRPIX2': size/2 + 0.5,
        'CRVAL2': dec,
        'NAXIS2': size
    }
    w = WCS(wcs_input_dict)
    
    return img_name, w

In [ ]:
for sga_id in np.unique(good_edge_spirals_axis_in_cluster['SGA_ID']):
    
    tf_list = good_edge_spirals_axis_in_cluster[good_edge_spirals_axis_in_cluster['SGA_ID'] == sga_id]
    center_list = centers_in_cluster[centers_in_cluster['SGA_ID'] == sga_id]
    
    try:
        targetid = int(center_list['TARGETID'][0])
    except TypeError as err:
        print(err)
        # print(sga_galaxy['TARGETID'])
        continue
    
    ra, dec, z = float(SGA['RA'][SGA_dict[sga_id]]), float(SGA['DEC'][SGA_dict[sga_id]]), float(SGA['Z_DESI'][SGA_dict[sga_id]])
    
    # D26 in arcmin
    d26 = SGA['D26'][SGA_dict[sga_id]]
    
    # Padd the image cutout of the galaxy.
    # Multiply by 60 (to arcsec), divide by 180 to get pixscale.
#     pixscale = 1.05*d26*60/180
    npix = np.minimum(int(1.025 * d26*60/0.262), 512)
    
    #print(targetid, sga_id, ra, dec)
#     img_file = get_cutout(targetid, ra, dec, size=npix, verbose=True)
    img_file, wcs = get_cutout(targetid, ra, dec, size=npix, verbose=True)
    img = mpl.image.imread(img_file)

    fig1 = plt.figure(figsize=(7,5))

    ax = fig1.add_subplot(111, projection=wcs)
    ax.imshow(np.flip(img, axis=0))
    ax.set(xlabel='ra', ylabel='dec')
    ax.text(int(0.02*npix), int(0.85*npix), 'TARGETID: {}\nSGA_ID: {}\n$z={{{:.4f}}}$'.format(targetid, sga_id, z), fontsize=9, color='yellow')
    overlay = ax.get_coords_overlay('icrs')
    overlay.grid(color='white', ls='dotted');

    # Add the location of the DESI fibers.
    # SDSS fibers are 2" diameter, DESI is 107 um with 70 um/" plate scale.
    r1 = SphericalCircle((ra * u.deg, dec * u.deg), (107./70) * u.arcsec,
                         edgecolor='black', facecolor='none', alpha=0.8, lw=3,
                         transform=ax.get_transform('icrs'))
    r2 = SphericalCircle((ra * u.deg, dec * u.deg), (107./70) * u.arcsec,
                         edgecolor='red', facecolor='none', alpha=0.8, lw=2,
                         transform=ax.get_transform('icrs'))
    ax.add_patch(r1)
    ax.add_patch(r2)

    for tft in tf_list:
        ra, dec = tft['RA'], tft['DEC']
        
        edgecolor2 = 'orange'
#         if tft['Z'] > 0.05:
#             edgecolor2 = 'orange'
#         else:
#             edgecolor2 = 'lime'
        
        # Add the location of the DESI fibers.
        # SDSS fibers are 2" diameter, DESI is 107 um with 70 um/" plate scale.
        r1 = SphericalCircle((ra * u.deg, dec * u.deg), (107./70) * u.arcsec,
                             edgecolor='lightcoral', facecolor='none', alpha=1, lw=3,
                             transform=ax.get_transform('icrs'))
        r2 = SphericalCircle((ra * u.deg, dec * u.deg), (107./70) * u.arcsec,
                             edgecolor=edgecolor2, facecolor='none', alpha=0.8, lw=2,
                             transform=ax.get_transform('icrs'))
        ax.add_patch(r1)
        ax.add_patch(r2)
        
        ax.text(ra, dec, str(tft['TARGETID']), transform=ax.get_transform('icrs'), color='white')
    
    fig1.subplots_adjust(top=0.85, right=0.85, bottom=0.15, left=0.15)
    
    fig1.savefig('cache/iron_VI_cutouts/dist_{}.png'.format(sga_id), dpi=120)
    
    fig1.clear()
    plt.close(fig1)
    '''
    #----------------------------------------------------------------------------------
    # Get spectra
    #----------------------------------------------------------------------------------
    coadds = get_spectra_for_sga(good_edge_spirals_axis_inComa, sga_id)
    n = coadds.num_spectra()

    fig2, axes = plt.subplots(n,1, figsize=(8,4*n), sharex=True, sharey=True, tight_layout=True)

    for i in range(n):
        wave = coadds.wave['brz']
        flux = coadds.flux['brz'][i]
        smoothed = gaussian_filter1d(flux, 7)

        ax = axes[i]
        ax.plot(wave, smoothed, label='TARGETID: {}'.format(coadds.fibermap['TARGETID'][i]))
        ax.set(ylabel=r'flux [$10^{-17}$ erg cm$^{-2}$ s$^{-1}$ $\AA^{-1}$]')
        if i+1 == n:
            ax.set(xlabel=r'$\lambda_\mathrm{obs}$ [$\AA$]')

        ax.legend(loc='upper right', fontsize=10)
    
    # figure = PdfPages('cache/DM_cutouts/dist_{}.pdf'.format(sga_id))
    # figure.savefig(fig1, dpi = 300)
    # figure.savefig(fig2, dpi = 120)
    # figure.close()
    
    break
    ''';